# Cirq Tutorials

With cirq we only have the function to convert a cirq circuit to a QLM circuit, and no providers nor algorithms, because cirq
doesn't offer the interface for that.

Cirq has a special feature allowing users to put arbitrary real powers of gates into circuits.

Not all gate support all powers in this conversion:


Gates supporting arbitrary real powers (all of these gates daggers and controls are also supported in power mode):
- X, Y, Z
- RX, RY, RZ
- XX, YY, ZZ
- RXX, RYY, RZZ
- PH
- S, T

Gates supporting integer powers (all of these gates daggers and controls are also supported in power mode):
- H
- SWAP (also supports square roots +/- 0.5)
- iSWAP


## Cirq to qlm conversion

Below we use an example showing the usage of arbitrary powers 


In [ ]:
import cirq

from numpy import sqrt, pi

from qat.interop.cirq.converters import cirq_to_qlm
from qat.core.util import get_syntax

gcirq = cirq.Circuit()
qreg = [cirq.LineQubit(i) for i in range(5)]

gcirq.append(cirq.X(qreg[0]) ** -3.67)
gcirq.append(cirq.Y(qreg[0]) ** 7.9)
gcirq.append(cirq.Z(qreg[0]) ** (sqrt(5)))
gcirq.append(cirq.S(qreg[0]) ** -pi)
gcirq.append(cirq.T(qreg[0]) ** (sqrt(7)-pi))
gcirq.append(cirq.SWAP(qreg[0], qreg[1]) ** -0.5)
gcirq.append(cirq.ISWAP(qreg[0], qreg[1]) ** 16.0)
for qbit in qreg:
    gcirq.append(cirq.measure(qbit))
    
# get result with included measures
qlm_circuit = cirq_to_qlm(gcirq)

for index, op in enumerate(qlm_circuit.ops):
    print("Gate {} with params {} on qubits {} and cbits {}".format(*get_syntax(qlm_circuit, index), op.cbits))

# get result with separated measures
print("---------------------------------------------------------")
qlm_circuit, to_measure = cirq_to_qlm(gcirq, sep_measures=True)

for index, op in enumerate(qlm_circuit.ops):
    print("Gate {} with params {} on qubits {}".format(*get_syntax(qlm_circuit, index)))
print("qubits to measure {}".format(to_measure))


## QLM to Cirq conversion

The following is an example on how to convert a QLM circuit to a Cirq circuit

In [ ]:
from qat.interop.cirq.converters import qlm_to_cirq
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
cirq_circuit = qlm_to_cirq(qlm_circuit)

print(cirq_circuit)